In [38]:
# import libraries needed
from pathlib import Path
import os, yaml, time, sys
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from numpy import fft
import scipy.stats as stats
#from .utils import *
# load in data
file_name = '../../Data/CubicBox_(-250.0, 250.0)_(-250.0, 250.0)_(-250.0, 250.0).npz'
particle_subsample = np.load(file_name)
x, y, z = particle_subsample['x'], particle_subsample['y'], particle_subsample['z']
vx, vy, vz = particle_subsample['vx'], particle_subsample['vy'], particle_subsample['vz'] 
x_range,y_range,z_range=[min(x),max(x)],[min(y),max(y)],[min(z),max(z)]

Functions written by Rianna for linear reconstruction

In [39]:
# rianna's code
# called in the function below for the velocity field
def wave_num(kgrid):
    """
    A function that computes the magnitude of the angular wave number k from it's cartesian components on a grid

    :param kgrid: The k-space meshgrid
    """
    kx, ky = kgrid[0], kgrid[1]
    kz = np.zeros_like(kx) if len(kgrid)==2 else kgrid[2]
    return np.sqrt(kx ** 2 + ky ** 2 + kz ** 2)

In [40]:
# rianna's code
# called in the function below for the velocity field
def get_velocity_grids(field, edges):
    """
    A function that returns the grid over which the velocity field will be computed in both real and fourier 
    space. 

    :param field:    The grid corresponding to the matter density field (2D or 3D)
    :param edges:    The side-length of the grid cells used to compute the velocity field.
    """
    # compute the number of dimensions of the density_field 
    N = len(np.shape(field))
    if N not in [2, 3]: 
        raise ValueError("Density field does not appear to be either a 2D or 3D field. The density field has a shape {}".format(np.shape(field))) 
    # compute the side length of the density field (in Mpc) 
    L = np.max(edges) - np.min(edges)
    ngrid = len(edges) - 1
    # kgrid_arrays = [np.fft.fftfreq(ngrid, L / (ngrid-1)) for _ in range(N-1)]
    # kgrid_arrays.extend([np.fft.rfftfreq(ngrid, L / (ngrid-1))])
    # kgrid_arrays = [np.fft.fftfreq(ngrid, L / (ngrid-1)) for _ in range(N)]
    kgrid_arrays = [2 * np.pi * np.fft.fftfreq(ngrid, L / (ngrid)) for _ in range(N)]
    # generate the fourier-space velocity grid
    kgrid = np.meshgrid(*reversed(kgrid_arrays)) if N==2 else np.meshgrid(*kgrid_arrays)
    return kgrid

In [41]:
# rianna's code
# computes the linear velocity field
def compute_linear_velocity_field(delta, grid_edges, H0=67, omega_m=0.3):
    """
    A function that computes the linear peculiar velocity field across a grid, using a set cosmology
    and a given matter density field.

    :param delta:   The input matter density field (2D or 3D)
    :param grid_edges:    The edges of the bins used to compude the density field (1D array)
    :param H0:              The hubble constant used in the velocity field computation (default is 67
                            km/s/Mpc from Plank2018)
    :param omega_m:         The cosmological matter density parameter used in the velocity field 
                            computation (default is 0.3 from Plank2018)
    """
    print('Computing the linear velocity field...')
    # compute f
    f = omega_m ** 0.6
    # compute the number of dimensions of the density field 
    N = len(np.shape(delta))
    # compute the real space and fourier space grids needed to compute the velocity field.
    k_grid = get_velocity_grids(delta, grid_edges)
    # # compute the wavenumber at each grid point
    k = wave_num(k_grid)
    # compute the k-space density field
    # delta_k = fft.rfftn(delta)
    delta_k = fft.fftn(delta)
    # initialise the fourier space velocity array
    velocity_kx = np.zeros_like(delta_k)
    velocity_ky = np.zeros_like(delta_k)
    velocity_kz = np.zeros_like(delta_k)
    # create the mask for the non-zero wavenumbers
    mask = k != 0
    velocity_kx[mask] = 1j * f * H0 * delta_k[mask] * k_grid[0][mask] / k[mask] ** 2
    velocity_ky[mask] = 1j * f * H0 * delta_k[mask] * k_grid[1][mask] / k[mask] ** 2
    if N == 3: 
        velocity_kz[mask] = 1j * f * H0 * delta_k[mask] * k_grid[2][mask] / k[mask] ** 2
    # compute the inverse transformation to get the real space velocity field
    # vx = np.fft.irfftn(velocity_ky, delta.shape)
    # vy = np.fft.irfftn(velocity_kx, delta.shape)
    # vz = np.fft.irfftn(velocity_kz, delta.shape) if N == 3 else np.zeros_like(vx)
    vx = np.real(np.fft.ifftn(velocity_ky, delta.shape))
    vy = np.real(np.fft.ifftn(velocity_kx, delta.shape))
    vz = np.real(np.fft.ifftn(velocity_kz, delta.shape) if N == 3 else np.zeros_like(vx))
    return [2*vx,2*vy,2*vz]

Functions for comparisons

In [42]:
# create initial histogram splitting the data
# returns the bins x bins x bins density histogram and 3 x bins histogram edges
def density_histogram(bin_num):
    combined=np.column_stack((x,y,z))
    hist,edges=np.histogramdd(combined, bins=[bin_num,bin_num,bin_num])
    return hist,edges

In [43]:
# create histogram for overdensity
# returns the bins x bins x bins overdensity histogram and 3 x bins histogram edges
def overdensity_histogram(bin_num):
    hist,edges=density_histogram(bin_num)
    smolboxvol=int(500/bin_num)**3
    densities=hist/smolboxvol
    otherav=len(x)/(500**3)
    overdense=densities/otherav-1
    return overdense,edges

In [44]:
# create arrays for which bin each particle belongs in
# returns 3 arrays for the x, y and z box corresponding to each particle
def binned_positions(bin_num,edges):
    x_indices=np.digitize(x,edges[0])-1
    y_indices=np.digitize(y,edges[1])-1
    z_indices=np.digitize(z,edges[2])-1
    bin_x = np.clip(x_indices, 0, bin_num-1)
    bin_y = np.clip(y_indices, 0, bin_num-1)
    bin_z = np.clip(z_indices, 0, bin_num-1)
    return bin_x,bin_y,bin_z

Main code

In [45]:
# obtain the reconstructed velocities
# the main value to assign first
bin_num=64

# create overdensity field
overdense,edges=overdensity_histogram(bin_num)

# get binned particles
bin_x,bin_y,bin_z=binned_positions(bin_num,edges)

# compute linear velocity field
rx,ry,rz=compute_linear_velocity_field(overdense,edges[0])

# assign reconstructed values to all corresponding boxes
bin_rvx=rx[bin_x,bin_y,bin_z]
bin_rvy=ry[bin_x,bin_y,bin_z]
bin_rvz=rz[bin_x,bin_y,bin_z]

# calculate difference between reconstructed velocity and true velocity for particle
bin_dx=abs(bin_rvx-vx)
bin_dy=abs(bin_rvy-vy)
bin_dz=abs(bin_rvz-vz)

Computing the linear velocity field...


In [51]:
# create a plot of particles, their distribution, and the reconstructed velocity in a certain cell
# select cell to examine
cell1,cell2,cell3=52,31,12

# cut the data
mask=(bin_x==cell1)&(bin_y==cell2)&(bin_z==cell3)
cell_particles=vy[mask]/100

# compute mean and standard deviation in the cell
cell_mean=np.mean(cell_particles)*100
cell_std=np.std(cell_particles)*100
cell_particles=cell_particles*100

# construct gaussian for the cell
cell_range=np.linspace(cell_mean-4*cell_std,cell_mean+4*cell_std,100)
pdf = stats.norm.pdf(cell_range,cell_mean,cell_std)

# select corresponding reconstructed velocity
cell_recon=ry[cell1,cell2,cell3]

# plot particles, distribution and different means
plt.fill_between(cell_range,np.zeros(100),pdf,color='b',alpha=0.5,label='Gaussian of velocities')
plt.hist(cell_particles,bins=25,density=True,color='r',alpha=0.5,label='Individual velocities')
plt.axvline(x=cell_recon,color='k',label='Reconstructed mean')
plt.axvline(x=cell_mean,color='b',label='Cell mean')
plt.xlabel('Velocity (km/s)')
plt.ylabel('Normalised frequency')
plt.legend()

# save figure
path = '../Figures/ComparisonPlots/Y velocities in cell ({:.0f},{:.0f},{:.0f})'.format(cell1,cell2,cell3)
plt.savefig(path, dpi=300)
plt.close()

In [47]:
# create a plot of the distribution of difference in x, y and z velocities
# histogram of log of each component
plt.hist(np.log10(bin_dx+1),bins=25,color='r',alpha=0.5,label='vx')
plt.hist(np.log10(bin_dy+1),bins=25,color='g',alpha=0.5,label='vy')
plt.hist(np.log10(bin_dz+1),bins=25,color='b',alpha=0.5,label='vz')
plt.xlabel('$\log_{10} \Delta v$')
plt.ylabel('Frequency')
plt.legend()

# save figure
path = '../Figures/ComparisonPlots/Delta velocity histogram'
plt.savefig(path, dpi=300)
plt.close()

<>:6: SyntaxWarning: invalid escape sequence '\l'
<>:6: SyntaxWarning: invalid escape sequence '\l'
/tmp/ipykernel_741/60729161.py:6: SyntaxWarning: invalid escape sequence '\l'
  plt.xlabel('$\log_{10} \Delta v$')


In [48]:
# retrieve mean and standard deviation in the difference between actual and reconstructed velocities
# select cell
cell1,cell2,cell3=14,60,8

# split data
mask=(bin_x==cell1)&(bin_y==cell2)&(bin_z==cell3)
cell_particles=bin_dx[mask]/100

# find the corresponding reconstructed velocity
cell_recon=rz[cell1,cell2,cell3]

#calculate mean and standard deviation in cell
cell_mean=np.mean(cell_particles)*100
cell_std=np.std(cell_particles)*100
cell_particles=cell_particles*100

# this should be editted for plotting purposes